In [1]:
import json
import itertools
from fractions import Fraction
import numpy as np
import pandas

#GUI elements
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

In [2]:
%run logic.ipynb
%run participant.ipynb


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [3]:
def update_dist(change):
    
    dropdown = dist_accordion.children[dist_accordion.selected_index].children[1]
    
    if dropdown.value != 'Custom':
        
        active_sliders = dist_accordion.children[dist_accordion.selected_index].children[2].children
        shape_slider = dist_accordion.children[dist_accordion.selected_index].children[3]

        values = distMap[dropdown.value](len(active_sliders),shape_slider.value)

        for slider in active_sliders:

            index = active_sliders.index(slider)
            slider.value = np.round(values[index], decimals=sliders_precision)
    #else:
        #TODO: reset to custom values
        

In [4]:
def dropdown_action(change):
    dropdown = dist_accordion.children[dist_accordion.selected_index].children[1]
    shape_slider = dist_accordion.children[dist_accordion.selected_index].children[3]
    shape_slider.hold_trait_notifications()

    # Update the shape slider according to the chosen distribution
    if(dropdown.value is 'Uniform' or dropdown.value is 'Custom'):
        shape_slider.disabled=True
        shape_slider.layout.visibility='hidden'
    else:

        if(dropdown.value is 'Geom'):
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.max = 1.
            shape_slider.value = 0.5
        elif(dropdown.value is 'Logser'):
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.max = 1. - shape_slider.step
            shape_slider.value = 0.6
        elif(dropdown.value is 'Planck'):
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.max = 1.
            shape_slider.value = 0.51
        elif(dropdown.value is 'Zipf'):
            shape_slider.max = 10.
            shape_slider.step = 0.01
            shape_slider.min = 1. + shape_slider.step
            shape_slider.value = 1.5

        shape_slider.disabled=False
        shape_slider.layout.visibility='visible'

In [5]:
#Example JSON
root_question = {'qid':0,
'question': 'Blood type?',
'answers': ['O+','A+','Other'],
'probability': ['1/3','1/3','1/3']}

follow_up_1 = {'qid':1,
'question': 'Specify...',
'answers': ['B+','AB+','O-','A-','Other'],
'probability': ['1/5','1/5','1/5','1/5','1/5']}

follow_up_2 = {'qid':2,
'question': 'Specify further...',
'answers': ['B-','AB-'],
'probability': ['1/2','1/2']}

paths = [[0,'Other',1], [1,'Other',2]]

json_poll = {
    'roots': [root_question],
    'children': [follow_up_1, follow_up_2],
    'paths':paths,
    'truth':['1/4']
}
json_poll

{'children': [{'answers': ['B+', 'AB+', 'O-', 'A-', 'Other'],
   'probability': ['1/5', '1/5', '1/5', '1/5', '1/5'],
   'qid': 1,
   'question': 'Specify...'},
  {'answers': ['B-', 'AB-'],
   'probability': ['1/2', '1/2'],
   'qid': 2,
   'question': 'Specify further...'}],
 'paths': [[0, 'Other', 1], [1, 'Other', 2]],
 'roots': [{'answers': ['O+', 'A+', 'Other'],
   'probability': ['1/3', '1/3', '1/3'],
   'qid': 0,
   'question': 'Blood type?'}],
 'truth': ['1/4']}

In [6]:
%run gui_components.ipynb

<IPython.core.display.Javascript object>

IndentationError: expected an indented block (<ipython-input-6-91d5aa547f1e>, line 4)

In [7]:
# Distribution sliders per question (non-flattened)
children = []

subtrees = pollToSubtrees(json_poll)
for question in subtrees:
    for subquestion in subtrees[question]:
        dist_accordion.set_title(subquestion['qid'], 'Question: '+subquestion['question'])
        html = widgets.HTML('<h3>Choose weight distribution...</h3>')
        to_add = []
        
        count = 0
        for answer in subquestion['answers']:
            probability = subquestion['probability'][count]
            to_add.append(widgets.FloatSlider(
                value=probability,
                min=0,
                max=1.0,
                step=0.01,
                description=answer+':',
                disabled=False,
                continuous_update=False,
                orientation='vertical',
                readout=True,
                readout_format='.1f',
            ))
            count +=1
        
        #Components
        _shape_slider = widgets.FloatSlider(min=0., max=5., step=0.1, value=0., continuous_update=False, description='Shape', disabled=True) 
        _dropdown = widgets.Dropdown(options=dists, value='Custom', description='Distribution:', disabled=False)
        _button = widgets.Button(description='Update weights', icon='fa-bar-chart', button_style='success')
        
        #Actions
        _button.on_click(update_dist)
        _dropdown.observe(dropdown_action)
        
        contents =[html, _dropdown, widgets.HBox(to_add),_shape_slider, _button]
        children.append( widgets.VBox(contents))
        
dist_accordion.children = children
dist_box = widgets.VBox([dist_accordion])

In [8]:
display(widgets.HTML('<center><h1> Choose Parameters'))
display(start_tab)

HTML(value=u'<center><h1> Choose Parameters')

VGFiKGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShMYWJlbCh2YWx1ZT11J0lucHV0IEpTT046JyksIFNlbGVjdChvcHRpb25zPSgnYmxvb2QuanNvbicsKSwgdmFsdWXigKY=


In [9]:
display(widgets.HTML('<center><h1> Set <i>True</i> Answer Distribution</h1></center>'))
display(dist_accordion)
dropdown_action(None)

HTML(value=u'<center><h1> Set <i>True</i> Answer Distribution</h1></center>')

QWNjb3JkaW9uKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIVE1MKHZhbHVlPXUnPGgzPkNob29zZSB3ZWlnaHQgZGlzdHJpYnV0aW9uLi4uPC9oMz4nKSwgRHJvcGRvd24oZGVzY3JpcHRpb27igKY=


In [10]:
participants = []
n = 100000
participant_budget = 40

matrices = pollToMatrix(json_poll)
epsilons = pollToEpsilon(json_poll)

#TODO: Loop x times for confidence

for i in range(0, n):
    new_participant = Participant(participant_budget, json_poll)
    new_participant.setMatrices(matrices)
    new_participant.setEpsilons(epsilons)
    participants.append(new_participant)
    


In [11]:
#TODO: read weights from dist_accordion

weights = pollToPathWeights(json_poll)
ps = pollToPaths(json_poll)

ans_dist_zip = {}
for question in ps.keys():
    paths = ps[question]
    weight = weights[question]
    zipped = []
    for path in paths:
         zipped.append((path, weight[pathToKey(path)]))
    ans_dist_zip[question] = zipped

#participant.createAnswer(ans_dist_zip)

In [12]:
#TODO: set answer dist

#TODO: run RR

#TODO: compare to ground truth

#TODO: display graphs